# Publishing Feedback

### Setup

In [1]:
# You can set them inline
import os
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Adding Feedback to a Run

You can add feedback to an existing Run programmatically as long as you know the run_id. Let's grab a run_id from the LangSmith UI and add it here.

In [3]:
run_id = "06dff15c-aee9-42db-bbba-fb9f7ee0ab56"

Now, let's add some continuous feedback

In [4]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-continuous",
  score=7.0,
  comment="This is a sample piece of continuous feedback",
)

Feedback(id=UUID('8d715bc5-e7ef-4643-ad70-9cb9ac4fc710'), created_at=datetime.datetime(2025, 1, 15, 4, 6, 10, 624301, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 1, 15, 4, 6, 10, 624301, tzinfo=datetime.timezone.utc), run_id=UUID('06dff15c-aee9-42db-bbba-fb9f7ee0ab56'), trace_id=None, key='sample-continuous', score=7.0, value=None, comment='This is a sample piece of continuous feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

And now, let's add some categorical feedback too!

In [5]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-categorical",
  value="no",
  comment="This is a sample piece of categorical feedback",
)

Feedback(id=UUID('8c1cd6ca-3e45-4d0b-9164-cc36f06428ab'), created_at=datetime.datetime(2025, 1, 15, 4, 6, 18, 141135, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 1, 15, 4, 6, 18, 141135, tzinfo=datetime.timezone.utc), run_id=UUID('06dff15c-aee9-42db-bbba-fb9f7ee0ab56'), trace_id=None, key='sample-categorical', score=None, value='no', comment='This is a sample piece of categorical feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-Generating Run IDs for Feedback

![Generate_run_id](../../images/generate_run_id.png)

Using LangChain, we offer the ability to pre-generate and define run IDs, before your code is invoked and the run ID is generated. With this functionality, you're able to access your run ID before initial generation, which can be useful for actions like sending feedback. The example below demonstrates this.

In [6]:
import uuid

pre_defined_run_id = uuid.uuid4()
pre_defined_run_id

UUID('81f7fc0e-0df4-4bf1-80c9-51cf4077ba05')

In [7]:
from langsmith import traceable

@traceable
def foo():
    return "This is a sample Run!"

We are passing in a config with our function call through `langsmith_extra` that contains our pre-defined run_id

In [8]:
foo(langsmith_extra={"run_id": pre_defined_run_id})

'This is a sample Run!'

Now we can directly create feedback on this run!

In [9]:
from langsmith import Client

client = Client()

client.create_feedback(pre_defined_run_id, "user_feedback", score=1)

Feedback(id=UUID('738b7f66-984b-46ac-ba1e-ba0c4a178656'), created_at=datetime.datetime(2025, 1, 15, 4, 8, 52, 418484, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 1, 15, 4, 8, 52, 418484, tzinfo=datetime.timezone.utc), run_id=UUID('81f7fc0e-0df4-4bf1-80c9-51cf4077ba05'), trace_id=None, key='user_feedback', score=1, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-signed Feedback URLs

![presigned url](../../images/presigned_url.png)


This can also be helpful for pre-signed feedback URLs. You would want to use these when you can't expose API keys or other secrets to the client, e.g. in a web application. Using a pre-determined run_id LangSmith has an endpoint create_presigned_feedback_token which will create a URL for sending feedback, without the use of secrets required.

In [10]:
pre_signed_url_id = uuid.uuid4()
pre_signed_url_id

UUID('27a8c91d-02b8-4f63-b9d1-92b4273f9e1b')

In [11]:
pre_signed_url = client.create_presigned_feedback_token(pre_signed_url_id, "user_presigned_feedback")

print(pre_signed_url)

id=UUID('8c4d5212-3340-4065-990c-e337bad5b304') url='https://api.smith.langchain.com/feedback/tokens/8c4d5212-3340-4065-990c-e337bad5b304' expires_at=datetime.datetime(2025, 1, 15, 7, 11, 6, 978163, tzinfo=datetime.timezone.utc)


Here, we can see that even though we haven't created a run yet, we're still able to generate the feedback URL.

Now, let's invoke our chain so the run with that ID is created:

In [12]:
foo(langsmith_extra={"run_id": pre_signed_url_id})

'This is a sample Run!'

Then, once our run is created, we can use the feedback URL to send feedback:

In [13]:
import requests

url_with_score = f"{pre_signed_url.url}?score=1"

response = requests.get(url_with_score)

if response.status_code >= 200 and response.status_code < 300:
    print("Feedback submitted successfully!")
else:
    print("Feedback submission failed!")

Feedback submitted successfully!
